# Fine Tune Distractor

# Pretrained

In [ ]:
!pip install transformers

In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_Destractive_model():
  tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
  model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")
  return model, tokenizer

In [ ]:
def pretrained_distractor(model, tokenizer, context, question, answer):
  input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
  inputs = tokenizer(input_text, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=128)
  distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
  distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
  distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]
  options = [answer] + distractors

  if "" or '' in options:
    try:
      options.remove("")
    except:
      options.remove('')

  random.shuffle(options)
  return question, options, answer

In [ ]:
#test
context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")
question = "What is the best title for the passage?"
answer = "Djokovic's application for special permission to enter the United States"

model, tokenizer = load_Destractive_model()
question, options, answer = pretrained_distractor(model, tokenizer, context, question, answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
print(question)
print(answer)
print(options)

What is the best title for the passage?
Djokovic's application for special permission to enter the United States
["Djokovic's preparation for the Miami Open", "Djokovic's application for special permission to enter the United States", "Djokovic's preparation for the Miami Open", "Djokovic's preparation for the Miami Open"]


# Inference code for Mohamed Essam

In [ ]:
!pip install transformers

In [ ]:
#import
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random

#GPU -> this for hole project
if torch.cuda.is_available():
    torch_device = 'cuda'
else:
    torch_device = 'cpu'
torch_device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#VIP model folder https://drive.google.com/drive/folders/1Y7B3EBI7TcK7VvppdFrUJ97BQwxhKHx_?usp=sharing choose 30000
def load_Distractor_model(Distractor_path = "", distractor_model_type = "t5-small", max_length = 512):
  distractor_model = AutoModelForSeq2SeqLM.from_pretrained(distractor_model_type)
  if torch_device == "cuda":
      distractor_model.cuda()
      state = torch.load(Distractor_path) # path to Distractor Gen (t5-large)
  else:
      state = torch.load(Distractor_path, map_location=torch.device('cpu'))

  model_state_dict = state['model']
  distractor_model.load_state_dict(model_state_dict)
  distractor_model.eval()
  print('DistractorGeneration Model loaded:', Distractor_path)

  distractor_tokenizer = AutoTokenizer.from_pretrained(distractor_model_type, model_max_length=max_length)
  distractor_tokenizer.add_special_tokens({"sep_token": "<sep>"})

  return distractor_tokenizer

distractor_tokenizer = load_Distractor_model(Distractor_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/t5-small-Race-Distractor-Generation-version0-step54920.pt", distractor_model_type = "t5-small", max_length = 512)
#VIP distractor_tokenizer --> path to

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

DistractorGeneration Model loaded: /content/drive/MyDrive/Distractor_Finetune/save_dir/t5-small-Race-Distractor-Generation-version0-step54920.pt


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:

context = r"""
There was a king who gave his daughter a beautiful diamond necklace. The necklace was stolen and his people in the kingdom searched everywhere but could not find it. So the king asked to search for it and put a reward for$50,000 for anyone who found it. One day, a clerk was walking home along a river when he saw something shinning in the river and when he looked, he saw the diamond necklace. He decided to try and catch it to get the $50,000 reward. He put his hand in the river and tried to catch the necklace, but somehow missed it and didn't catch it. He took his hand out and looked again and the necklace was still there. He tried again. But strangely, he still missed the necklace! He came out and started walking away, feeling sad. Just then, an old man walking by saw him, and asked him what was the matter. The clerk didn't want to share the secret with him, thinking he might take the necklace for himself, so he refused to tell him anything. The old man could see this man was troubled. He again asked the clerk to tell him the problem and promised that he would not tell anyone about it. The clerk told him about the necklace and how he tried and tried to catch it, but kept failing. The old man then told him that perhaps he should try looking upward, toward the branches of the tree, instead of in the river. The clerk looked up and true enough, the necklace was hanging on the branch of a tree. He had been trying to take a reflection of the real necklace all this time..""".replace("\n", "")

question = "Where did the clerk find the necklace at last?		"
answer = "On the tree."


In [ ]:
def generate_distractors(t5_tokenizer, context = "",question = "" ,answer = "" ,separator='<sep>', max_length=512, torch_device='cpu',):
  #output ==> dis <sep> dis <> dis
  input_text = question + ' ' + separator + ' ' + answer + ' ' + separator + ' ' + context
  encoding = t5_tokenizer(
      [input_text],
      padding="longest",
      max_length=max_length,
      truncation=True,
      return_tensors="pt",)

  input_ids = encoding.input_ids

  if torch_device == 'cuda':
      input_ids = input_ids.cuda()


  outputs = distractor_model.generate(
  input_ids,
  max_new_tokens=128,
  do_sample=True,)

  distractors = distractor_tokenizer.decode(outputs[0], skip_special_tokens=False)
  distractors = distractors.replace(distractor_tokenizer.pad_token, "").replace(distractor_tokenizer.eos_token, "")
  distractors = [y.strip() for y in distractors.split(distractor_tokenizer.sep_token)]
  #[gg,gg,gg]
  options = [answer] + distractors

  if "" or '' in options:
    try:
      options.remove("")
    except:
      options.remove('')

  random.shuffle(options)
  return question, options, answer

prefix = ['A', 'B' , 'C', 'D']
question, options, answer = generate_distractors(distractor_tokenizer, context = context,question = question ,answer = answer ,separator='<sep>', max_length=512, torch_device= torch_device,)
print(f"Q: {question} \n ")
for i , option in zip(prefix, options):
  print(f"  {i}: {option} \n")
print(f"correct: {answer}")



NameError: name 'distractor_model' is not defined

In [ ]:
#Test load Model
# model2: distractor generation

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

max_length = 512
Distractor_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/t5-small-Race-Distractor-Generation-version0-step54920.pt"
distractor_model_type = "t5-small"

distractor_model = AutoModelForSeq2SeqLM.from_pretrained(distractor_model_type)
if torch_device == "cuda":
    distractor_model.cuda()
    state = torch.load(Distractor_path) # path to Distractor Gen (t5-large)
else:
    state = torch.load(Distractor_path, map_location=torch.device('cpu'))
model_state_dict = state['model']
distractor_model.load_state_dict(model_state_dict)
distractor_model.eval()

distector_tokenizer = AutoTokenizer.from_pretrained(distractor_model_type, model_max_length=max_length)
distector_tokenizer.add_special_tokens({"sep_token": ""})


0

# FineTune

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# https://drive.google.com/drive/folders/1jYwVMvCnogWhGv5ULIFft9c4_oUPQ-Qk?usp=sharing

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!python /content/drive/MyDrive/Distractor_Finetune/train.py

# trials

In [ ]:
#Distractor Generation
distractor_tokenizer = AutoTokenizer.from_pretrained(distractor_model_type, model_max_length=max_length)
distractor_tokenizer.add_special_tokens({"sep_token": "<sep>"})

def prepare_distractor_input(
    t5_tokenizer,
    context,
    question,
    answer,
    separator='<sep>',
    max_length=512,
    torch_device='cpu',):

    """
    input: question <sep> answer <sep> article
    output: distractor1 <sep> distractor2 <sep> distractor3
    """
    input_text = question + ' ' + separator + ' ' + answer + ' ' + separator + ' ' + context
    encoding = t5_tokenizer(
        [input_text],
        padding="longest",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",)

    input_ids = encoding.input_ids

    if torch_device == 'cuda':
        input_ids = input_ids.cuda()

    return input_ids


distractor_input_ids = prepare_distractor_input(
        distractor_tokenizer,
        context = context, # doc_x, sum_y
        question = question,
        answer = answer,
        separator = distractor_tokenizer.sep_token,
        torch_device = torch_device
)

outputs = distractor_model.generate(
    distractor_input_ids,
    max_new_tokens=128,
    do_sample=True,
)

distractors = distractor_tokenizer.decode(outputs[0], skip_special_tokens=False)
print(distractors)
distractors = distractors.replace(distractor_tokenizer.pad_token, "").replace(distractor_tokenizer.eos_token, "")
print(distractors)
distractors = [y.strip() for y in distractors.split(distractor_tokenizer.sep_token)]
print(distractors)
options = [answer] + distractors
print(options)


NameError: name 'AutoTokenizer' is not defined

In [ ]:
def prepare_answering_input(
    tokenizer, # longformer_tokenizer
    question,
    options,
    context,
    max_seq_length=4096,
    torch_device='cpu',
):
    """
    this currently only supports longformer
    """
    c_plus_q = context + ' ' + tokenizer.bos_token + ' ' + question
    c_plus_q_4 = [c_plus_q] * len(options)

    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)

    if torch_device == 'cuda':
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()

    example_encoded = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }


In [ ]:
#UPCOMING

# sampled_question_option_examples.append((question, options))
# Stage3: Multiple-Choice Answering PROBABILITY
answering_max_len = 128
longformer_model = 'allenai/longformer-large-4096'
answering_max_len = 4096
from transformers import LongformerTokenizer, LongformerForMultipleChoice

answering_model = LongformerForMultipleChoice.from_pretrained(longformer_model)
if torch_device == "cuda":
    answering_model.cuda()
    state = torch.load(answering_model_path)
else:
    state = torch.load(answering_model_path, map_location=torch.device('cpu'))
model_state_dict = state['model']
answering_model.load_state_dict(model_state_dict)
answering_model.eval()
print('Answering Model loaded:', answering_model_path)

answering_tokenizer = LongformerTokenizer.from_pretrained(longformer_model)
answering_given_inputs = prepare_answering_input(
    tokenizer=answering_tokenizer,
    question=question,
    options=options,
    context=context,
    max_seq_length=answering_max_len,
    torch_device=torch_device
)

answering_outputs = answering_model(**answering_given_inputs)
probs = torch.softmax(answering_outputs['logits'], dim=-1)[0].cpu().tolist()
probs = ["{:.6f}".format(p) for p in probs]

print("---------------------------------------------------------------------------------------")

# USE Distractive pretrained Model

In [ ]:
!pip install transformers

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")

In [ ]:
context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")
question = "What is the best title for the passage?"
answer = "Djokovic's application for special permission to enter the United States"


In [ ]:
import random

In [ ]:
input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=128)
distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]
options = [answer] + distractors

if "" or '' in options:
  try:
    options.remove("")
  except:
    options.remove('')

random.shuffle(options)

In [ ]:
options

["Djokovic's preparation for the Miami Open",
 "Djokovic's application for special permission to enter the United States",
 "Djokovic's preparation for the Miami Open",
 "Djokovic's preparation for the Miami Open"]

# Drafts

## Load Dataset

In [ ]:
!tar -xzvf "/content/drive/MyDrive/NTI_datasets/RACE.tar.gz" -C "/content/drive/MyDrive/NTI_datasets/"     #[run this cell to extract tar.gz files]

## Fine_tune_fromHugging 2

In [ ]:
t5_model = 't5-small'
save_dir = "model_weights/"
model_name = f"{t5_model}-Race-Distractor-Generation-version0"

lr0          = 5e-5
batch_size   = 8
num_workers  = 0
num_epochs   = 10
max_length   = 512
valid_step   = 5000

In [ ]:
from datasets import load_dataset

dataset = load_dataset("race", "all")
dataset["train"][10]

In [ ]:
train_data = RaceDistractorGeneration(
        tokenizer = t5_tokenizer,
        data_split = "train",
        shuffle_distractors = True,
        separator = t5_tokenizer.sep_token,
    )

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

## Adjust code TO retrain if it interrupts

In [ ]:
import numpy as nd
# Load the weights
if force_restart or not weights_path.exists():
    print("\nStarting the training of T5-model from scratch\n")
    model.save(weights_path)
    train_loss = []
    validate_loss = []

else:
    print("\nLoading weights at %s" % weights_path)
    model.load(weights_path, optimizer)

    train_load_losses_np = np.load(train_loss_path)
    train_loss = train_load_losses_np.tolist()

    validate_load_losses_np = np.load(validate_loss_path)
    validate_loss = validate_load_losses_np.tolist()
    #assert (len(train_loss) == (model.step/157)) , "validate loss != train loss "

    print("T5-model weights loaded from step %d" % model.step)
    print(f"\ntrain loss loaded from: {train_loss_path}, length = {len(train_loss)}")
    print(f"\nvalidate loss loaded from: {validate_loss_path}, length = {len(validate_loss)}")
    assert (len(validate_loss) == len(train_loss)) , "validate loss != train loss "



In [ ]:
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from datasets import load_dataset

class RaceQuestionAnswerGeneration(Dataset):
    def __init__(self, tokenizer, data_split, separator='<sep>'):
        """
        task:
            - input: article (i.e. context)
            - output: question <sep> answer
        args:
            tokenizer: tokenizer
            data_split: train, validation, test
        """
        data = load_dataset("race", "all", split=data_split)
        self.data = data
        self.tokenizer = tokenizer
        self.separator = separator
        self.label_mapping = {label: i for i, label in enumerate(["A", "B", "C", "D"])}
        print("RaceQuestionAnswerGeneration Initialized")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        # example_id = example["example_id"]
        question = example["question"]
        context = example["article"]
        options = example["options"]
        label_example = example["answer"]
        answer = options[self.label_mapping[label_example]]

        # input & output
        input = context
        output = question + ' ' + self.separator + ' ' + answer
        return {'input': input, 'output': output}

class RaceDistractorGeneration(Dataset):
    def __init__(self, tokenizer, data_split, shuffle_distractors=False, separator='<sep>'):
        """
        task:
            - input: question <sep> answer <sep> article
            - output: distractor1 <sep> distractor2 <sep> distractor3
        args:
            tokenizer: tokenizer
            data_split: train, validation, test
        """
        data = load_dataset("race", "all", split=data_split)
        self.data = data
        self.tokenizer = tokenizer
        self.separator = separator
        self.label_mapping = {label: i for i, label in enumerate(["A", "B", "C", "D"])}
        self.all_labels = [0, 1, 2, 3]
        self.shuffle_distractors = shuffle_distractors
        print("RaceQuestionAnswerGeneration Initialized")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        # example_id = example["example_id"]
        question = example["question"]
        context = example["article"]
        options = example["options"]
        label_example = example["answer"]
        answer_i = self.label_mapping[label_example]
        answer = options[answer_i]
        distractor_ids = [i for i in self.all_labels if i != answer_i]
        if self.shuffle_distractors:
            random.shuffle(distractor_ids)
        distractors = [options[i] for i in distractor_ids]

        # input & output
        input = question + ' ' + self.separator + ' ' + answer + ' ' + self.separator + ' ' + context
        output = distractors[0] + ' ' + self.separator + ' ' + distractors[1] + ' ' + self.separator + ' ' + distractors[2]
        return {'input': input, 'output': output}

class RaceAnsweringModel(Dataset):
    def __init__(self,
            data_split,
        ):
        """
        """
        data = load_dataset("race", "all", split=data_split)
        self.data = data
        self.label_mapping = {label: i for i, label in enumerate(["A", "B", "C", "D"])}
        self.all_labels = [0, 1, 2, 3]
        print("RaceAnsweringModel Initialized")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        question = example["question"]
        context = example["article"]
        options = example["options"]
        label_example = example["answer"]
        answer_i = self.label_mapping[label_example]

        return {'context': context, 'question': question, 'options': options, 'answer_i': answer_i}

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained('t5-small', model_max_length=512)
t5_tokenizer.add_special_tokens({"sep_token": "<sep>"})

# ---------------------------- Data ---------------------------- #
train_data = RaceDistractorGeneration(
    tokenizer = t5_tokenizer,
    data_split = "train",
    shuffle_distractors = True,
    separator = t5_tokenizer.sep_token,
)
print("len_train_data:", len(train_data))

valid_data = RaceDistractorGeneration(
    tokenizer = t5_tokenizer,
    data_split = "validation",
    separator = t5_tokenizer.sep_token,
)
print("len_valid_data:", len(valid_data))


In [ ]:
print(len(train_data))
batch_size = 8
total_iters = len(train_data)
steps_per_epoch = np.ceil(total_iters / batch_size).astype(np.int32)

print(len(valid_data))

In [ ]:
import os
import sys
import random
import numpy as np

from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from data_loader import RaceDistractorGeneration

force_restart = True

t5_model = 't5-small'
model_name = f"{t5_model}-Race-Distractor-Generation-version0"

save_dir = "/content/drive/MyDrive/Distractor_Finetune/save_dir"

weights_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/t5_model.pt"
load_weights_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/t5-small-Race-Distractor-Generation-version0-step5000.pt"
train_loss_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/train_loss.npy"
validate_loss_path = "/content/drive/MyDrive/Distractor_Finetune/save_dir/validate_loss.npy"

lr0          = 5e-5
batch_size   = 32
num_workers  = 0
num_epochs   = 10
max_length   = 512
valid_step   = 5000

t5_tokenizer = AutoTokenizer.from_pretrained(t5_model, model_max_length=max_length)
t5_tokenizer.add_special_tokens({"sep_token": "<sep>"})

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("torch_device:", torch_device)
print("model_name:", model_name)
print("t5_model:", t5_model)
print("lr0:", lr0)
print("batch_size:", batch_size)
print("num_workers:", num_workers)
print("num_epochs:", num_epochs)
print("valid_step:", valid_step)
print("max_length:", max_length)


def experiment():
    # ---------------------------- Data ---------------------------- #
    train_data = RaceDistractorGeneration(
        tokenizer = t5_tokenizer,
        data_split = "train",
        shuffle_distractors = True,
        separator = t5_tokenizer.sep_token,
    )
    print("len_train_data:", len(train_data))
    train_loader = torch.utils.data.DataLoader(
                    train_data,
                    batch_size=batch_size,
                    num_workers=num_workers,
                    shuffle=True,
                    collate_fn=collate_fn)

    valid_data = RaceDistractorGeneration(
        tokenizer = t5_tokenizer,
        data_split = "validation",
        separator = t5_tokenizer.sep_token,
    )
    print("len_valid_data:", len(valid_data))
    valid_loader = torch.utils.data.DataLoader(
                    valid_data,
                    batch_size=batch_size,
                    num_workers=num_workers,
                    shuffle=False,
                    collate_fn=collate_fn)
    # ---------------------------- Model ---------------------------- #

    model = AutoModelForSeq2SeqLM.from_pretrained(t5_model)

    if force_restart or not weights_path.exists():
      print("\nStarting the training of T5-model from scratch\n")
      train_loss = []
      validate_loss = []

      train_loss_as_numpy = np.asarray(train_loss)
      np.save(train_loss_path , train_loss_as_numpy)
      validate_loss_as_numpy = np.asarray(validate_loss)
      np.save(validate_loss_path , validate_loss_as_numpy)

      if torch_device == "cuda":
          model.cuda()
      print("#parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
      # by default, it's not training!!!
      model.train()
    else:
      print("\nLoading weights at %s" % load_weights_path)
      model.load(weights_path, optimizer)

      train_load_losses_np = np.load(train_loss_path)
      train_loss = train_load_losses_np.tolist()

      validate_load_losses_np = np.load(validate_loss_path)
      validate_loss = validate_load_losses_np.tolist()

      print("T5-model weights loaded from step %d" % model.step)
      print(f"\ntrain loss loaded from: {train_loss_path}, length = {len(train_loss)}")
      print(f"\nvalidate loss loaded from: {validate_loss_path}, length = {len(validate_loss)}")
      assert (len(validate_loss) == len(train_loss)) , "validate loss != train loss "

    # ----------------- Optimizer and Loss Function ----------------- #
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr0,betas=(0.9,0.999),eps=1e-08,weight_decay=0)
    optimizer.zero_grad()
    training_step = 0
    stop_counter = 0
    best_val_loss = 99999999
    total_iters = len(train_data)
    steps_per_epoch = np.ceil(total_iters / batch_size).astype(np.int32)

    for epoch_i in range(num_epochs):
        batches_loss = 0
        train_counter = 0
        for iter_, sample in enumerate(train_loader):
            if sample is None:
                continue

            input_ids, attention_mask = sample['input_ids'], sample['attention_mask']
            labels = sample['labels']

            if torch_device == 'cuda':
                input_ids = input_ids.cuda()
                attention_mask = attention_mask.cuda()
                labels = labels.cuda()

            loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
            loss.backward()
            curr_batch_size = len(labels)
            batches_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()

            if training_step % 1 == 0:
                print("{}, Epoch: {}/{}  |  {}/{}  |  loss = {:.8f}".format(str(datetime.now()), epoch_i, num_epochs, training_step, steps_per_epoch, loss))
                sys.stdout.flush()

            if training_step % valid_step == 0:
                state = {
                    'training_step': training_step,
                    'model': model.state_dict(),
                }
                savepath = "{}/{}-step{}.pt".format(save_dir, model_name, training_step)
                torch.save(state, savepath)
                print("Saved at {}".format(savepath))

                model.eval()
                with torch.no_grad():
                    valid_loss = validation(model, valid_loader)
                print("Valid Loss = {:.6f}".format(valid_loss))

                train_loss.append((batches_loss)/len(train_counter))
                validate_loss.append((valid_loss))


                model.train()




                if valid_loss < best_val_loss:
                    stop_counter = 0
                    best_val_loss = valid_loss
                    print("Model improved".format(stop_counter))
                else:
                    stop_counter += 1
                    print("Model not improved #{}".format(stop_counter))
                    if stop_counter == 3:
                        print("Stop training!")
                        return

            training_step += 1

        print("finish epoch: {}".format(epoch_i+1))

    print("Finish Training")

def validation(model, valid_loader):
    valid_loss = 0
    counter = 0
    for sample in valid_loader:
        input_ids, attention_mask = sample['input_ids'], sample['attention_mask']
        labels = sample['labels']

        if torch_device == 'cuda':
            input_ids = input_ids.cuda()
            attention_mask = attention_mask.cuda()
            labels = labels.cuda()

        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
        valid_loss += loss.item()
        counter += 1
        if counter % 50 == 0:
            print("#", end="")
            sys.stdout.flush()
    print()
    return valid_loss / counter

def collate_fn(list_of_items):
    """
    each item is a dictionary:
    """
    list_of_items = [x for x in list_of_items if x is not None]
    batch_size = len(list_of_items)
    if batch_size == 0: return None

    input_sequences, output_sequences = [], []
    for item in list_of_items:
        input_sequences.append(item['input'])
        output_sequences.append(item['output'])

    encoding = t5_tokenizer(
        input_sequences,
        padding="longest",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
    )
    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    target_encoding = t5_tokenizer(
        output_sequences,
        padding="longest",
        max_length=max_length,
        truncation=True,
    )

    # the forward function automatically creates the correct decoder_input_ids
    labels = target_encoding.input_ids
    # replace padding token id's of the labels by -100 so it's ignored by the loss
    labels = torch.tensor(labels)
    labels[labels == t5_tokenizer.pad_token_id] = -100

    return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
    }

if __name__ == "__main__":
    experiment()